# Fire Air Quality Analysis

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import os
import scipy.stats as st
from scipy.stats import linregress


## Import Data
Make sure that CSVs have been created in `Resources` directory prior to running

In [4]:
fire_data = pd.read_csv('./Resources/Fires.csv', dtype={'FIPS_NAME': 'str',
                                                        'COMPLEX_NAME': 'str',
                                                        'FIRE_NAME': 'str',
                                                        'FIRE_CODE': 'str'
                                                       })
fire_data.head()

In [4]:
# Filter down to CA data only
ca_fire_data = fire_data.loc[fire_data['STATE'] == 'CA' ]
ca_fire_data.head()

,OBJECTID,FOD_ID,FPA_ID,FIRE_YEAR,FIPS_NAME,COMPLEX_NAME,FIRE_NAME,FIRE_SIZE,DISCOVERY_DATE,FIRE_CODE,STAT_CAUSE_DESCR,CONT_DATE,STATE,LATITUDE,LONGITUDE,DISCOVERY_TIME,CONT_TIME
0,1,1,FS-1418826,2005,Plumas,NaN,FOUNTAIN,0.10,2453403.5,BJ8K,Miscellaneous,2453403.5,CA,40.036944,-121.005833,1300.0,1730.0
1,2,2,FS-1418827,2004,Placer,NaN,PIGEON,0.25,2453137.5,AAC0,Lightning,2453137.5,CA,38.933056,-120.404444,845.0,1530.0
2,3,3,FS-1418835,2004,El Dorado,NaN,SLACK,0.10,2453156.5,A32W,Debris Burning,2453156.5,CA,38.984167,-120.735556,1921.0,2024.0
3,4,4,FS-1418845,2004,Alpine,NaN,DEER,0.10,2453184.5,NaN,Lightning,2453189.5,CA,38.559167,-119.913333,1600.0,1400.0
4,5,5,FS-1418847,2004,Alpine,NaN,STEVENOT,0.10,2453184.5,NaN,Lightning,2453189.5,CA,38.559167,-119.933056,1600.0,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015CAIRS29019636,2015,NaN,NaN,ODESSA 2,0.01,2457291.5,NaN,Missing/Undefined,2457291.5,CA,40.481637,-122.389375,1726.0,1843.0
1880461,1880462,300348373,2015CAIRS29217935,2015,NaN,NaN,NaN,0.20,2457300.5,NaN,Miscellaneous,NaN,CA,37.617619,-120.938570,126.0,NaN
1880462,1880463,300348375,2015CAIRS28364460,2015,NaN,NaN,NaN,0.10,2457144.5,NaN,Missing/Undefined,NaN,CA,37.617619,-120.938570,2052.0,NaN
1880463,1880464,300348377,2015CAIRS29218079,2015,NaN,NaN,NaN,2.00,2457309.5,NaN,Missing/Undefined,NaN,CA,37.672235,-120.898356,2309.0,NaN


In [6]:
ca_air_data = pd.read_csv('./Resources/CA_air_quality_annual_summary.csv')
ca_air_data

,state_code,county_code,site_num,parameter_code,poc,latitude,longitude,datum,parameter_name,sample_duration,...,seventy_five_percentile,fifty_percentile,ten_percentile,local_site_name,address,state_name,county_name,city_name,cbsa_name,date_of_last_change
0,6,27,20,61104,1,36.057185,-117.861369,NAD83,Wind Direction - Resultant,1 HOUR,...,227.9,132.9,33.0,Coso Gate,COSO GATE,California,Inyo,NaN,"Bishop, CA",2020-05-21
1,6,27,20,61101,1,36.057185,-117.861369,NAD83,Wind Speed - Scalar,1 HOUR,...,12.6,8.0,2.3,Coso Gate,COSO GATE,California,Inyo,NaN,"Bishop, CA",2020-05-21
2,6,27,20,62201,1,36.057185,-117.861369,NAD83,Relative Humidity,1 HOUR,...,35.0,25.0,14.0,Coso Gate,COSO GATE,California,Inyo,NaN,"Bishop, CA",2020-05-21
3,6,27,20,61106,1,36.057185,-117.861369,NAD83,Std Dev Hz Wind Direction,1 HOUR,...,32.9,18.1,8.0,Coso Gate,COSO GATE,California,Inyo,NaN,"Bishop, CA",2020-05-21
4,6,27,20,61106,1,36.057185,-117.861369,NAD83,Std Dev Hz Wind Direction,1 HOUR,...,31.8,17.7,8.1,Coso Gate,COSO GATE,California,Inyo,NaN,"Bishop, CA",2020-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305644,6,79,2007,61106,1,35.046730,-120.587770,NAD83,Std Dev Hz Wind Direction,1 HOUR,...,33.0,20.4,12.0,Arroyo Grande CDF,"2391 Willow Road, Arroyo Grande, California",California,San Luis Obispo,Arroyo Grande,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",2020-05-21
305645,6,79,2007,88101,1,35.046730,-120.587770,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,...,11.8,8.3,3.7,Arroyo Grande CDF,"2391 Willow Road, Arroyo Grande, California",California,San Luis Obispo,Arroyo Grande,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",2020-05-21
305646,6,79,2007,88101,1,35.046730,-120.587770,NAD83,PM2.5 - Local Conditions,24-HR BLK AVG,...,12.2,8.4,3.0,Arroyo Grande CDF,"2391 Willow Road, Arroyo Grande, California",California,San Luis Obispo,Arroyo Grande,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",2020-05-21
305647,6,79,2007,88101,1,35.046730,-120.587770,NAD83,PM2.5 - Local Conditions,1 HOUR,...,11.0,7.0,2.0,Arroyo Grande CDF,"2391 Willow Road, Arroyo Grande, California",California,San Luis Obispo,Arroyo Grande,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",2020-05-21
